# Workshop 001: Proceso ETL de Datos de Candidatos

Universidad Autónoma de Occidente

Maestría en Inteligencia Artificial y Ciencia de Datos

Procesamiento y Extracción de Información

**Estudiante:** Juan Diego Díaz Guzmán

**Código de Estudiante:** 22500222

**Profesor:** Javier Alejandro Vergara

**Fecha:** 20/02/2025

## Introducción

Este nootebook implementa un proceso ETL, para analizar los candidatos de proceso de selección. El proyecto incluye las siguientes etapas:

1. **Extracción**: Lectura de datos desde un archivo CSV que contiene información de 50,000 candidatos.
2. **Transformación**: Procesamiento y limpieza de datos, incluyendo:
   - Normalización de campos
   - Cálculo de métricas de contratación
   - Categorización de candidatos
3. **Carga**: Almacenamiento de datos en una base de datos PostgreSQL para su posterior análisis.


# Importación de Librerías

In [1]:
import yaml
from datetime import datetime
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql

# Configuración de visualización de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 20)

print("Librerías importadas exitosamente!")


Librerías importadas exitosamente!


# Lectura y Exploración del CSV

In [2]:
try:
    # ruta del archivo CSV
    csv_path = '../data/candidates.csv'
    
    print(f"Leyendo archivo: {csv_path}")
    
    # Lectura del CSV
    df = pd.read_csv(csv_path, 
                     delimiter=';',
                     encoding='utf-8')
    
    print("\nArchivo leído exitosamente!")
    
except Exception as e:
    print(f"Error al leer el archivo: {e}")
    
#Exploración inicial de datos
if 'df' in locals():
    print("\nDimensiones del DataFrame:")
    print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
    
    print("\nColumnas del DataFrame:")
    print(df.columns.tolist())
    
    print("\nPrimeras 5 filas:")
    print(df.head())
    
    print("\nInformación del DataFrame:")
    print(df.info())
    
    print("\nEstadísticas descriptivas:")
    print(df.describe())
    
    print("\nValores nulos por columna:")
    print(df.isnull().sum())

#Validación básica de datos
def validate_data(df):
    """
    Realiza validaciones básicas en los datos.
    """
    validations = {
        "Registros totales": len(df),
        "Columnas esperadas presentes": all(col in df.columns for col in [
            'First Name', 'Last Name', 'Email', 'Application Date', 'Country',
            'YOE', 'Seniority', 'Technology', 'Code Challenge Score', 'Technical Interview Score'
        ]),
        "Emails únicos": df['Email'].nunique(),
        "Rango de scores válido": (
            (df['Code Challenge Score'].between(0, 10)) & 
            (df['Technical Interview Score'].between(0, 10))
        ).all()
    }
    
    print("\nResultados de la validación:")
    for check, result in validations.items():
        print(f"{check}: {result}")

Leyendo archivo: ../data/candidates.csv

Archivo leído exitosamente!

Dimensiones del DataFrame:
Filas: 50000, Columnas: 10

Columnas del DataFrame:
['First Name', 'Last Name', 'Email', 'Application Date', 'Country', 'YOE', 'Seniority', 'Technology', 'Code Challenge Score', 'Technical Interview Score']

Primeras 5 filas:
   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                 

# Funciones de Base de Datos

In [3]:
def load_config(file_path="config.yaml"):
    """
    Carga la configuración desde el archivo YAML.
    
    Args:
        file_path (str): Ruta al archivo de configuración
        
    Returns:
        dict: Configuración cargada
    """
    try:
        with open(file_path, "r") as file:
            return yaml.safe_load(file)
    except Exception as e:
        print(f"Error al cargar la configuración: {e}")
        return None

def get_db_connection():
    """
    Crea una conexión a la base de datos PostgreSQL.
    
    Returns:
        connection: Objeto de conexión a PostgreSQL
    """
    config = load_config()
    if not config:
        return None
    
    try:
        # Obtener configuración de la base de datos
        db_config = config["database"]
        
        # Crear conexión
        conn = psycopg2.connect(
            dbname=db_config["name"],
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=db_config["port"]
        )
        
        return conn
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Probar la conexión
try:
    conn = get_db_connection()
    if conn:
        print("Conexión a la base de datos establecida exitosamente!")
        conn.close()
except Exception as e:
    print(f"Error: {e}")

Conexión a la base de datos establecida exitosamente!


**Creación de Estructura en Base de Datos**

En esta sección crearemos la tabla en PostgreSQL para almacenar los datos de los candidatos. 
La tabla contendrá los siguientes campos:
- First Name
- Last Name
- Email
- Application Date
- Country
- YOE (Years of Experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview Score

**Creación de la Tabla**

In [4]:
def create_candidates_table():
    """
    Crea la tabla candidates en la base de datos.
    """
    create_table_query = """
    CREATE TABLE IF NOT EXISTS candidates (
        id SERIAL PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(150),
        application_date DATE,
        country VARCHAR(100),
        years_of_experience INTEGER,
        seniority VARCHAR(50),
        technology VARCHAR(100),
        code_challenge_score DECIMAL(4,2),
        technical_interview_score DECIMAL(4,2),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    
    conn = None
    try:
        # Obtener conexión
        conn = get_db_connection()
        conn.autocommit = True
        
        # Crear cursor y ejecutar query
        with conn.cursor() as cur:
            cur.execute(create_table_query)
            print("Tabla 'candidates' creada exitosamente!")
            
    except Exception as e:
        print(f"Error al crear la tabla: {e}")
    finally:
        if conn:
            conn.close()

# Crear la tabla
create_candidates_table()

Tabla 'candidates' creada exitosamente!


**Carga de Datos**

In [5]:
def insert_candidates_data(df, batch_size=1000):
    """
    Inserta los datos en la tabla candidates en lotes.
    
    Args:
        df (pandas.DataFrame): DataFrame con los datos a insertar
        batch_size (int): Tamaño del lote para inserción
    """
    # Query de inserción
    insert_query = """
    INSERT INTO candidates (
        first_name, last_name, email, application_date, country,
        years_of_experience, seniority, technology,
        code_challenge_score, technical_interview_score
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """
    
    conn = None
    try:
        # Obtener conexión
        conn = get_db_connection()
        
        # Crear cursor
        with conn.cursor() as cur:
            # Procesar datos en lotes
            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i + batch_size]
                
                # Preparar datos para inserción
                values = [tuple(x) for x in batch.values]
                
                # Ejecutar inserción en lote
                cur.executemany(insert_query, values)
                
                # Commit del lote
                conn.commit()
                
                print(f"Insertados registros {i} a {i + len(batch)}")
                
        print("¡Todos los datos han sido insertados exitosamente!")
        
    except Exception as e:
        print(f"Error al insertar datos: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

# Preparar los datos para la inserción
def prepare_data_for_insert(df):
    """
    Prepara los datos para la inserción en la base de datos.
    """
    try:
        # Convertir la fecha de aplicación al formato correcto
        df['Application Date'] = pd.to_datetime(df['Application Date'])
        
        # Convertir años de experiencia a entero
        df['YOE'] = df['YOE'].astype(int)
        
        # Convertir scores a float
        df['Code Challenge Score'] = df['Code Challenge Score'].astype(float)
        df['Technical Interview Score'] = df['Technical Interview Score'].astype(float)
        
        return df
    except Exception as e:
        print(f"Error al preparar los datos: {e}")
        return None

# Ejecutar la preparación y carga de datos
df_prepared = prepare_data_for_insert(df)
if df_prepared is not None:
    insert_candidates_data(df_prepared)

Insertados registros 0 a 1000
Insertados registros 1000 a 2000
Insertados registros 2000 a 3000
Insertados registros 3000 a 4000
Insertados registros 4000 a 5000
Insertados registros 5000 a 6000
Insertados registros 6000 a 7000
Insertados registros 7000 a 8000
Insertados registros 8000 a 9000
Insertados registros 9000 a 10000
Insertados registros 10000 a 11000
Insertados registros 11000 a 12000
Insertados registros 12000 a 13000
Insertados registros 13000 a 14000
Insertados registros 14000 a 15000
Insertados registros 15000 a 16000
Insertados registros 16000 a 17000
Insertados registros 17000 a 18000
Insertados registros 18000 a 19000
Insertados registros 19000 a 20000
Insertados registros 20000 a 21000
Insertados registros 21000 a 22000
Insertados registros 22000 a 23000
Insertados registros 23000 a 24000
Insertados registros 24000 a 25000
Insertados registros 25000 a 26000
Insertados registros 26000 a 27000
Insertados registros 27000 a 28000
Insertados registros 28000 a 29000
Inserta

**Transformaciones y Análisis final**

In [6]:
def transform_and_load_data():
    """
    Aplica transformaciones y carga los datos en la nueva tabla.
    """
    # Query para obtener datos específicos (sin incluir el id ni created_at)
    select_query = """
    SELECT 
        first_name, last_name, email, application_date, country,
        years_of_experience, seniority, technology,
        code_challenge_score, technical_interview_score
    FROM candidates;
    """
    
    # Query de inserción corregido
    insert_query = """
    INSERT INTO candidates_transformed (
        first_name, last_name, email, application_date, country,
        years_of_experience, seniority, technology,
        code_challenge_score, technical_interview_score, hired
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    
    conn = None
    try:
        conn = get_db_connection()
        
        # Obtener datos originales
        with conn.cursor() as cur:
            cur.execute(select_query)
            rows = cur.fetchall()
            
            if not rows:
                print("No se encontraron datos en la tabla original")
                return
                
            print(f"Procesando {len(rows)} registros...")
            
            # Procesar cada registro
            for row in rows:
                # Los valores ya vienen en el orden correcto del select
                values = list(row)
                
                # Calcular el valor de hired
                code_score = float(values[8]) if values[8] is not None else 0
                tech_score = float(values[9]) if values[9] is not None else 0
                hired = code_score >= 7 and tech_score >= 7
                
                # Añadir el valor de hired al final
                values.append(hired)
                
                # Imprimir un ejemplo del primer registro para depuración
                if rows.index(row) == 0:
                    print("\nEjemplo de valores a insertar:")
                    print(values)
                
                # Insertar el registro
                cur.execute(insert_query, values)
            
            conn.commit()
            print(f"\nSe transformaron y cargaron {len(rows)} registros exitosamente!")
            
    except Exception as e:
        print(f"Error en la transformación y carga: {e}")
        print("Valores que causaron el error:", values if 'values' in locals() else "No disponible")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

# Limpiar la tabla transformada antes de insertar
def clear_transformed_table():
    conn = None
    try:
        conn = get_db_connection()
        with conn.cursor() as cur:
            cur.execute("TRUNCATE TABLE candidates_transformed RESTART IDENTITY;")
            conn.commit()
            print("Tabla transformada limpiada exitosamente")
    except Exception as e:
        print(f"Error al limpiar la tabla: {e}")
    finally:
        if conn:
            conn.close()

# Ejecutar el proceso completo
print("Reiniciando proceso de transformación...")
clear_transformed_table()
transform_and_load_data()

# Verificar los resultados
def verify_transformed_data():
    """
    Verifica los datos en la tabla transformada
    """
    conn = None
    try:
        conn = get_db_connection()
        with conn.cursor() as cur:
            # Total de registros
            cur.execute("SELECT COUNT(*) FROM candidates_transformed;")
            total = cur.fetchone()[0]
            
            # Candidatos contratados
            cur.execute("SELECT COUNT(*) FROM candidates_transformed WHERE hired = TRUE;")
            hired = cur.fetchone()[0]
            
            print(f"\nResultados de la transformación:")
            print(f"Total de registros: {total}")
            print(f"Candidatos contratados: {hired}")
            print(f"Porcentaje de contratación: {(hired/total)*100 if total > 0 else 0:.2f}%")
            
            # Distribución por tecnología
            print("\nContratados por tecnología:")
            cur.execute("""
                SELECT technology, COUNT(*) as total
                FROM candidates_transformed
                WHERE hired = TRUE
                GROUP BY technology
                ORDER BY total DESC
                LIMIT 5;
            """)
            for row in cur.fetchall():
                print(f"{row[0]}: {row[1]} contratados")
                
    except Exception as e:
        print(f"Error al verificar datos transformados: {e}")
    finally:
        if conn:
            conn.close()

verify_transformed_data()

Reiniciando proceso de transformación...
Tabla transformada limpiada exitosamente
Procesando 250000 registros...

Ejemplo de valores a insertar:
['Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', Decimal('3.00'), Decimal('3.00'), False]

Ejemplo de valores a insertar:
['Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', Decimal('3.00'), Decimal('3.00'), False]

Ejemplo de valores a insertar:
['Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', Decimal('3.00'), Decimal('3.00'), False]

Ejemplo de valores a insertar:
['Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', Decimal('3.00'), Decimal('3.00'), False]

Ejemplo de valores a insertar:
['Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway'

**Análisis Detallado y Visualizaciones**

In [7]:
def detailed_analysis():
    """
    Realiza un análisis detallado de los datos transformados
    """
    queries = {
        "Tasa de contratación por país": """
            SELECT 
                country,
                COUNT(*) as total_candidates,
                COUNT(*) FILTER (WHERE hired = TRUE) as hired_candidates,
                ROUND(
                    (COUNT(*) FILTER (WHERE hired = TRUE) * 100.0 / COUNT(*))::numeric,
                    2
                ) as hiring_rate
            FROM candidates_transformed
            GROUP BY country
            ORDER BY hiring_rate DESC
            LIMIT 10;
        """,
        
        "Tasa de contratación por tecnología": """
            SELECT 
                technology,
                COUNT(*) as total_candidates,
                COUNT(*) FILTER (WHERE hired = TRUE) as hired_candidates,
                ROUND(
                    (COUNT(*) FILTER (WHERE hired = TRUE) * 100.0 / COUNT(*))::numeric,
                    2
                ) as hiring_rate
            FROM candidates_transformed
            GROUP BY technology
            ORDER BY hiring_rate DESC
            LIMIT 10;
        """,
        
        "Tasa de contratación por años de experiencia": """
            SELECT 
                years_of_experience,
                COUNT(*) as total_candidates,
                COUNT(*) FILTER (WHERE hired = TRUE) as hired_candidates,
                ROUND(
                    (COUNT(*) FILTER (WHERE hired = TRUE) * 100.0 / COUNT(*))::numeric,
                    2
                ) as hiring_rate
            FROM candidates_transformed
            GROUP BY years_of_experience
            ORDER BY years_of_experience;
        """,
        
        "Tasa de contratación por seniority": """
            SELECT 
                seniority,
                COUNT(*) as total_candidates,
                COUNT(*) FILTER (WHERE hired = TRUE) as hired_candidates,
                ROUND(
                    (COUNT(*) FILTER (WHERE hired = TRUE) * 100.0 / COUNT(*))::numeric,
                    2
                ) as hiring_rate
            FROM candidates_transformed
            GROUP BY seniority
            ORDER BY hiring_rate DESC;
        """
    }
    
    conn = None
    try:
        conn = get_db_connection()
        with conn.cursor() as cur:
            for title, query in queries.items():
                print(f"\n{title}:")
                print("-" * 80)
                cur.execute(query)
                results = cur.fetchall()
                
                if "país" in title or "tecnología" in title:
                    print(f"{'Nombre':<20} {'Total':<10} {'Contratados':<12} {'Tasa (%)':<10}")
                    print("-" * 80)
                    for row in results:
                        print(f"{row[0]:<20} {row[1]:<10} {row[2]:<12} {row[3]:<10}")
                elif "experiencia" in title:
                    print(f"{'Años':<10} {'Total':<10} {'Contratados':<12} {'Tasa (%)':<10}")
                    print("-" * 80)
                    for row in results:
                        print(f"{row[0]:<10} {row[1]:<10} {row[2]:<12} {row[3]:<10}")
                else:
                    print(f"{'Nivel':<15} {'Total':<10} {'Contratados':<12} {'Tasa (%)':<10}")
                    print("-" * 80)
                    for row in results:
                        print(f"{row[0]:<15} {row[1]:<10} {row[2]:<12} {row[3]:<10}")
                        
    except Exception as e:
        print(f"Error en el análisis detallado: {e}")
    finally:
        if conn:
            conn.close()

# Ejecutar el análisis detallado
print("\nGenerando análisis detallado...")
detailed_analysis()


Generando análisis detallado...

Tasa de contratación por país:
--------------------------------------------------------------------------------
Nombre               Total      Contratados  Tasa (%)  
--------------------------------------------------------------------------------

Tasa de contratación por tecnología:
--------------------------------------------------------------------------------
Nombre               Total      Contratados  Tasa (%)  
--------------------------------------------------------------------------------

Tasa de contratación por años de experiencia:
--------------------------------------------------------------------------------
Años       Total      Contratados  Tasa (%)  
--------------------------------------------------------------------------------

Tasa de contratación por seniority:
--------------------------------------------------------------------------------
Nivel           Total      Contratados  Tasa (%)  
-------------------------------------

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

# Configuración de seaborn
sns.set_theme(style="whitegrid")

def read_and_validate_data(csv_path='../data/candidates.csv'):
    """
    Lee y valida los datos del CSV
    """
    try:
        # Leer el CSV
        print(f"Leyendo archivo: {csv_path}")
        df = pd.read_csv(csv_path, delimiter=';', encoding='utf-8')
        print("\nArchivo leído exitosamente!")
        
        # Exploración inicial
        print("\nDimensiones del DataFrame:")
        print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
        
        print("\nColumnas del DataFrame:")
        print(df.columns.tolist())
        
        print("\nPrimeras 5 filas:")
        print(df.head())
        
        print("\nInformación del DataFrame:")
        print(df.info())
        
        print("\nEstadísticas descriptivas:")
        print(df.describe())
        
        print("\nValores nulos por columna:")
        print(df.isnull().sum())
        
        # Validación de datos
        validations = {
            "Registros totales": len(df),
            "Columnas esperadas presentes": all(col in df.columns for col in [
                'First Name', 'Last Name', 'Email', 'Application Date', 'Country',
                'YOE', 'Seniority', 'Technology', 'Code Challenge Score', 'Technical Interview Score'
            ]),
            "Emails únicos": df['Email'].nunique(),
            "Rango de scores válido": (
                (df['Code Challenge Score'].between(0, 10)) & 
                (df['Technical Interview Score'].between(0, 10))
            ).all()
        }
        
        print("\nResultados de la validación:")
        for check, result in validations.items():
            print(f"{check}: {result}")
            
        # Agregar columna de contratados
        df['Hired'] = (df['Code Challenge Score'] >= 7) & (df['Technical Interview Score'] >= 7)
        
        return df
        
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

def plot_hiring_metrics(df):
    """Crear gráfico de métricas generales de contratación"""
    total_candidates = len(df)
    hired_candidates = df['Hired'].sum()
    hire_rate = (hired_candidates / total_candidates) * 100

    plt.figure(figsize=(10, 5))
    
    metrics = ['Total Candidatos', 'Candidatos Contratados', 'Tasa de Contratación (%)']
    values = [total_candidates, hired_candidates, hire_rate]
    
    ax = sns.barplot(x=metrics, y=values)
    
    for i, v in enumerate(values):
        ax.text(i, v, f'{v:,.1f}', ha='center', va='bottom')
    
    plt.title('Métricas de Contratación')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_technology_distribution(df):
    """Crear gráfico de distribución por tecnología"""
    plt.figure(figsize=(10, 6))
    tech_counts = df['Technology'].value_counts()
    
    plt.pie(tech_counts.values, labels=tech_counts.index, autopct='%1.1f%%')
    plt.title('Distribución por Tecnología')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

def plot_experience_vs_hiring(df):
    """Crear gráfico de experiencia vs contratación"""
    plt.figure(figsize=(12, 6))
    
    exp_hire = df.groupby('YOE').agg({
        'Hired': ['count', 'sum']
    }).reset_index()
    exp_hire.columns = ['YOE', 'Total', 'Hired']
    
    sns.barplot(x='YOE', y='Total', data=exp_hire, color='lightblue', label='No Contratados')
    sns.barplot(x='YOE', y='Hired', data=exp_hire, color='darkblue', label='Contratados')
    
    plt.title('Contrataciones por Años de Experiencia')
    plt.xlabel('Años de Experiencia')
    plt.ylabel('Número de Candidatos')
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_country_distribution(df):
    """Crear gráfico de distribución por país"""
    plt.figure(figsize=(12, 6))
    
    top_countries = df['Country'].value_counts().head(10)
    
    ax = sns.barplot(y=top_countries.index, x=top_countries.values, palette='viridis')
    
    for i, v in enumerate(top_countries.values):
        ax.text(v, i, f' {v:,}', va='center')
    
    plt.title('Top 10 Países de Origen')
    plt.xlabel('Número de Candidatos')
    plt.ylabel('País')
    plt.tight_layout()
    plt.show()

def plot_score_distribution(df):
    """Crear gráfico de distribución de puntajes"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    sns.histplot(data=df, x='Code Challenge Score', bins=20, ax=ax1)
    ax1.axvline(x=7, color='r', linestyle='--', label='Umbral de aprobación')
    ax1.set_title('Distribución de Code Challenge Scores')
    ax1.legend()
    
    sns.histplot(data=df, x='Technical Interview Score', bins=20, ax=ax2)
    ax2.axvline(x=7, color='r', linestyle='--', label='Umbral de aprobación')
    ax2.set_title('Distribución de Technical Interview Scores')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

def print_summary_statistics(df):
    """Imprimir estadísticas resumen"""
    print("\nEstadísticas Resumen:")
    print("-" * 50)
    print(f"Total de candidatos: {len(df):,}")
    print(f"Candidatos contratados: {df['Hired'].sum():,}")
    print(f"Tasa de contratación: {(df['Hired'].mean() * 100):.1f}%")
    print(f"\nPromedio de años de experiencia: {df['YOE'].mean():.1f} años")
    print(f"Mediana de años de experiencia: {df['YOE'].median():.1f} años")
    print("\nDistribución por Seniority:")
    print(df['Seniority'].value_counts().to_string())
    print("\nPuntajes promedio:")
    print(f"Code Challenge: {df['Code Challenge Score'].mean():.2f}")
    print(f"Technical Interview: {df['Technical Interview Score'].mean():.2f}")

def display_all_visualizations(df):
    """Mostrar todos los gráficos"""
    print_summary_statistics(df)
    plot_hiring_metrics(df)
    plot_technology_distribution(df)
    plot_experience_vs_hiring(df)
    plot_country_distribution(df)
    plot_score_distribution(df)

# Uso del código:
# 1. Primero leer y validar los datos
df = read_and_validate_data()

# 2. Si la lectura fue exitosa, mostrar las visualizaciones
if df is not None:
    display_all_visualizations(df)
else:
    print("No se pudieron cargar los datos. Por favor verifica la ruta del archivo.")

ModuleNotFoundError: No module named 'matplotlib'

In [12]:
pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.0 MB 18.1 MB/s eta 0:00:01
   ----------------------------- ---------- 6.0/8.0 MB 15.4 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 13.8 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.6 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
